In [1]:
import datetime
import numpy as np
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import talib as ta

In [2]:
win = pd.read_excel('INDICES.xlsx', sheet_name='WIN', date_parser='Data')

In [3]:
win = win.dropna()

In [4]:
win = win.sort_values('Data')

In [5]:
bop = ta.BOP(win.Open, win.Max, win.Min, win.Close)

In [6]:
rsi = ta.RSI(win.Close, 21)

In [7]:
win['Rsi'] = rsi

In [8]:
win['Bop'] = bop

In [9]:
win.Bop.min()

-1.0

In [176]:
# Parâmetros
tAlta = np.array([])
tBaixa = np.array([])
maxTend = 0
minTend = 0
tendencia = '0'
profit = pd.DataFrame(columns=['DataEnt','Entrada','DataSai','Saida', 'Operacao','Saldo'])
posicionado = 0 # 1 - Sim, 0 - Não
volMinOper = 500000000
vlrMinDifMedias = 80
compraBop = 0.10 # de -1 a 1
vendaBop = -0.10
compraRsi = 70 # de 0 a 100
vendaRsi = 30

In [11]:
def getMinValue(vlr1 : float, vlr2 : float):
    if(vlr1 <= vlr2):
        return vlr1
    else:
        return vlr2
    
def getMaxValue(vlr1 : float, vlr2 : float):
    if(vlr1 >= vlr2):
        return vlr1
    else:
        return vlr2    

In [177]:
for i in win.index:
    tBaixa = np.append(tBaixa, win.Close[i])
    tAlta = np.append(tAlta, win.Close[i])
    if(win.Media21[i] > win.Media9[i]): #identificando a tendencia a partir da diferença das medias
        if(tendencia == 'Baixa'): # identificar alternancia de tendencias
            tBaixa = np.append(tBaixa, win.Close[i])
            if(posicionado == 0):
                if(getMaxValue(win.Close[i], win.Open[i]) < getMinValue(minTend, maxTend) and 
                   (win.Media21[i] - win.Media9[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper and
                    win.Bop[i] < vendaBop and win.Rsi[i] < vendaRsi): #localizando oportunidade de compra a partir da comparacao do vlr atual e o fundo conhecido da tendencia
                    profit.loc[profit.index.size, ['DataEnt','Entrada','Operacao']] = win.Data[i], getMaxValue(win.Close[i], win.Open[i]), 'Venda'
                    posicionado = 1
                    minTend = min(tBaixa)
                else:
                    minTend = min(tBaixa)
            elif(posicionado == 1):
                if(win.Close[i] < getMinValue(minTend, maxTend) and 
                   (win.Media21[i] - win.Media9[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper and
                    win.Bop[i] < vendaBop and win.Rsi[i] < vendaRsi):
                    minTend = min(tBaixa)
                else:
                    profit.loc[profit.index.size -1, ['DataSai','Saida', 'Saldo']] = win.Data[i], getMinValue(win.Close[i], win.Open[i]), profit.Entrada[profit.index.size -1] - win.Close[i]
                    posicionado = 0
                    minTend = min(tBaixa)
        else:
            tendencia = 'Baixa'
            maxTend = min(tAlta)
            minTend = win.Close[i]
            tBaixa = []
            i = i-1
    elif(win.Media21[i] < win.Media9[i]):
        if(tendencia == 'Alta'): # identificar alternancia de tendencias
            tAlta = np.append(tAlta, win.Close[i])
            if(posicionado == 0):
                if(getMinValue(win.Close[i], win.Open[i]) > getMaxValue(minTend, maxTend) and 
                   (win.Media9[i] - win.Media21[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper and
                    win.Bop[i] > compraBop  and win.Rsi[i] < vendaRsi): #localizando oportunidade de compra a partir da comparacao do vlr atual e o fundo conhecido da tendencia
                    profit.loc[profit.index.size, ['DataEnt','Entrada','Operacao']] = win.Data[i], getMinValue(win.Close[i], win.Open[i]), 'Compra'
                    posicionado = 1
                    maxTend = max(tAlta)
                else:
                    maxTend = max(tAlta)
            elif(posicionado == 1):
                if(win.Close[i] > getMaxValue(minTend, maxTend) and 
                   (win.Media9[i] - win.Media21[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper and
                    win.Bop[i] > compraBop and win.Rsi[i] < vendaRsi):
                    maxTend = max(tAlta)
                else:
                    profit.loc[profit.index.size -1, ['DataSai','Saida', 'Saldo']] = win.Data[i], getMaxValue(win.Close[i], win.Open[i]), win.Close[i] - profit.Entrada[profit.index.size -1]
                    posicionado = 0
                    maxTend = max(tAlta)
        else:
            tendencia = 'Alta'
            maxTend = win.Close[i]
            minTend = max(tBaixa)
            tAlta = []
            i = i-1
        

In [178]:
profit.Saldo.sum()

860

In [168]:
profit[['Saldo']].query('Saldo > 0').count()

Saldo    12
dtype: int64

In [169]:
profit[['Saldo']].query('Saldo < 0').count()

Saldo    7
dtype: int64

In [143]:
profit[['Saldo']].query('Saldo > 0').sum()

Saldo    1460.0
dtype: float64

In [144]:
profit[['Saldo']].query('Saldo < 0').sum()

Saldo   -515.0
dtype: float64

In [145]:
profit[['Saldo']].query('Saldo == 0').count()

Saldo    1
dtype: int64

In [146]:
profit = profit.dropna()

In [148]:
profit.to_csv('teste.csv', index=False)

In [21]:
-1 < -10

False